# Research Track II - Enrico Fiasché
# User Interface
## Initialize node

In [1]:
import rospy
import jupyros as jr
import time
from rt2_assignment1.srv import Command
import actionlib
import rt2_assignment1.msg
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import ipywidgets as widgets
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from std_msgs.msg import Int32, Bool
from IPython.display import display, clear_output
from tf.transformations import euler_from_quaternion
import math 
%matplotlib widget

rospy.init_node('user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)

# action client used to cancel the goal
action_client = actionlib.SimpleActionClient("go_to_point",rt2_assignment1.msg.PositionAction)
if(action_client.wait_for_server()):
    print("Connected!")
else:
    print("Unable to connect with the action server")
    
lin_vel = 0.2
ang_vel = -3.0

Connected!


## Start and Stop random targets

In [2]:
start_sim = Button(description='START',
            layout=Layout(width='200', align="center", grid_area='b1'),
            style=ButtonStyle(button_color='lightgreen'))
stop_sim = Button(description='STOP',
            layout=Layout(width='200', align="center", grid_area='b2'),
            style=ButtonStyle(button_color='salmon'))

HBox([start_sim,stop_sim])

grid = GridBox(children=[start_sim, stop_sim],
        layout=Layout(
        width='40%',
        grid_template_rows='auto auto',
        grid_template_columns='33% 15% 33%',
        grid_template_areas=''' "b1 . b2 " ''')
        )

display(grid)
starting_time = 0
status = "idle"
def start(event):
    global starting_time
    
    ui_client("start")
    starting_time = rospy.Time.now()
    clear_output(wait=True)
    status = "started!"
    display(grid)
    display('Robot '+status)
    
def stop(event):
    action_client.cancel_all_goals()
    ui_client("stop")
    clear_output(wait=True)
    status = "stopped!"
    display(grid)
    display('Robot '+status)
    
start_sim.on_click(start)
stop_sim.on_click(stop)
display('Robot '+status)

GridBox(children=(Button(description='START', layout=Layout(grid_area='b1', width='200'), style=ButtonStyle(bu…

'Robot started!'

## Sliders

In [3]:
import ipywidgets as widgets

linear_slider = widgets.FloatSlider(value=0.2, min=0, max=0.6, step=0.1, description='linear_velocity')
angular_slider = widgets.FloatSlider(value=-3.0, min=-5.0, max=0, step=0.5, description='angular_velocity')

def on_linear_change(change):
    global lin_vel, ang_vel
    
    const_twist = Twist()
    lin_vel = change['new']
    const_twist.linear.x = lin_vel
    const_twist.angular.z = ang_vel
    pub_consts.publish(const_twist)
    
def on_angular_change(change):
    global lin_vel, ang_vel
    
    const_twist = Twist()
    ang_vel = change['new']
    const_twist.linear.x = lin_vel
    const_twist.angular.z = ang_vel
    pub_consts.publish(const_twist)

pub_consts = rospy.Publisher('/consts', Twist, queue_size=1)
linear_slider.observe(on_linear_change, names='value')
angular_slider.observe(on_angular_change, names='value')

widgets.HBox([widgets.VBox([linear_slider, angular_slider])])

## Controller

In [4]:
up_arrow = Button(description='^',
            layout=Layout(width='auto', align="center", grid_area='b1'),
            style=ButtonStyle(button_color='lightgreen'))
down_arrow = Button(description='v',
            layout=Layout(width='auto', align="center", grid_area='b5'),
            style=ButtonStyle(button_color='lightgreen'))
left_arrow = Button(description='<',
            layout=Layout(width='auto', align="center", grid_area='b2'),
            style=ButtonStyle(button_color='lightgreen'))
right_arrow = Button(description='>',
            layout=Layout(width='auto', align="center", grid_area='b4'),
            style=ButtonStyle(button_color='lightgreen'))
stopRobot_button = Button(description='STOP',
            layout=Layout(width='auto', align="center", grid_area='b3'),
            style=ButtonStyle(button_color='salmon'))

controller_grid = GridBox(children=[up_arrow,down_arrow,left_arrow,right_arrow,stopRobot_button],
        layout=Layout(
        width='40%',
        grid_template_rows='auto auto',
        grid_template_columns='25% 10% 20% 20% 20%',
        grid_template_areas='''
        ". . . b1 . "
        "b3 . b2 b5 b4"
        ''')
        )

def up(event):
    global lin_vel
    
    action_client.cancel_all_goals()
    ui_client("stop")
    twist_msg = Twist()
    twist_msg.linear.x = lin_vel
    twist_msg.angular.z = 0.0
    twist_pub.publish(twist_msg)

def down(event):
    global lin_vel
    
    action_client.cancel_all_goals()
    ui_client("stop")
    twist_msg = Twist()
    twist_msg.linear.x = -lin_vel
    twist_msg.angular.z = 0.0
    twist_pub.publish(twist_msg)

def left(event):
    global ang_vel
    
    action_client.cancel_all_goals()
    ui_client("stop")
    twist_msg = Twist()
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = ang_vel*0.3
    twist_pub.publish(twist_msg)
    
def right(event):
    global ang_vel
    
    action_client.cancel_all_goals()
    ui_client("stop")
    twist_msg = Twist()
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = -ang_vel*0.3
    twist_pub.publish(twist_msg)
    
def stopRobot(event):
    global lin_vel, ang_vel
    
    action_client.cancel_all_goals()
    ui_client("stop")
    twist_msg = Twist()
    twist_msg.linear.x = 0.0
    twist_msg.angular.z = 0.0
    twist_pub.publish(twist_msg)
    
display(controller_grid)
twist_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)

up_arrow.on_click(up)
down_arrow.on_click(down)
left_arrow.on_click(left)
right_arrow.on_click(right)
stopRobot_button.on_click(stopRobot)

GridBox(children=(Button(description='^', layout=Layout(grid_area='b1', width='auto'), style=ButtonStyle(butto…

# Graphical Information

## Cmd_vel vs Actual Velocity

In [4]:
fig_vel = plt.figure(1)
ax_lin = fig_vel.add_subplot(211)
ax_ang = fig_vel.add_subplot(212)

plot_cmd_lin, = ax_lin.plot([], [], label="lin_cmd")
plot_cmd_ang, = ax_ang.plot([], [], label="ang_cmd")
plot_odom_lin, = ax_lin.plot([], [], label="lin_odom")
plot_odom_ang, = ax_ang.plot([], [], label="ang_odom")

plots = [plot_cmd_lin, plot_cmd_ang, plot_odom_lin, plot_odom_ang]

ax_lin.set(xlabel='Time', ylabel='Amplitude')
ax_lin.set_title("Linear velocity")

ax_ang.set(xlabel='Time', ylabel='Amplitude')
ax_ang.set_title("Angular velocity")

ax_lin.legend()
ax_ang.legend()

time_cmd, time_odom, lin_cmd, ang_cmd, lin_odom, ang_odom = [], [], [], [], [], []

def cmd_callback(cmd_msg):
    global lin_cmd, ang_cmd, time_cmd
    
    lin_cmd.append(cmd_msg.linear.x)
    ang_cmd.append(cmd_msg.angular.z)
    
    if(len(time_cmd) == 0):
        time_cmd.append(0.05)
    else:
        time_cmd.append(time_cmd[len(time_cmd)-1]+0.05)
    

def odom_callback_velocity(odom_msg):
    global lin_odom, ang_odom, time_odom
    
    lin_odom.append(odom_msg.twist.twist.linear.x)
    ang_odom.append(odom_msg.twist.twist.angular.z)
    
    if(len(time_odom) == 0):
        time_odom.append(0.05)
    else:
        time_odom.append(time_odom[len(time_odom)-1]+0.05)

def plot_init():
   
    ax_lin.set_xlim(0,10)
    ax_lin.set_ylim(-0.1,0.8)
    
    ax_ang.set_xlim(0,10)
    ax_ang.set_ylim(-0.8,0.8)

    return plots,

def update_plot(frame):

    xmin, xmax = ax_lin.get_xlim()
    if len(time_cmd) and time_cmd[len(time_cmd)-1] > xmax:
        ax_lin.set_xlim(xmin+10,xmax+10)
        ax_lin.figure.canvas.draw()

    xmin, xmax = ax_ang.get_xlim()
    if len(time_odom) and time_odom[len(time_odom)-1] > xmax:
        ax_ang.set_xlim(xmin+10,xmax+10)
        ax_ang.figure.canvas.draw()

    plot_cmd_lin.set_data(time_cmd, lin_cmd)
    plot_cmd_ang.set_data(time_cmd, ang_cmd)
    
    plot_odom_lin.set_data(time_odom, lin_odom)
    plot_odom_ang.set_data(time_odom, ang_odom)
    
    return plots,

jr.subscribe('/cmd_vel', Twist, cmd_callback)
rospy.Subscriber('/odom', Odometry, odom_callback_velocity)

ani = FuncAnimation(fig_vel, update_plot, init_func=plot_init)
fig_vel.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot targets reached and cancelled

In [5]:
names = ["Reached","Cancelled"]
values = [0,0]
c = ["lightgreen","salmon"]

fig_targets = plt.figure(2) 
ax_targets = fig_targets.add_subplot(111)
bar_plot = ax_targets.bar(names,values, color=c)

ax_targets.set_title('Number of target reached and cancelled')
ax_targets.set(ylabel='Numbers')

def target_callback(msg):
    global values, bar_plot, ax_targets

    if(msg.data == 1):
        values[0] += 1

    elif(msg.data == 0):
        values[1] += 1
    
    bar_plot.remove()
    rospy.sleep(0.5)
    bar_plot = ax_targets.bar(names,values, color=c)

rospy.Subscriber('stateResult', Int32, target_callback)
fig_targets.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Time required to reach a target

In [4]:
num_bins = 100
times = []

fig_times = plt.figure(3)
ax_times = fig_times.add_subplot()
counts, bins, bars = ax_times.hist([], bins=num_bins)

ax_times.set_title("Time required to reach a target")
ax_times.set(xlabel="Time", ylabel="??")

def times_cb(msg):
    global starting_time, times, bars, ax_times, num_bins
    
    if(msg.data == 1):
        finish_time = rospy.Time.now()
        times.append(finish_time.to_sec() - starting_time.to_sec())
        print("Times"+str(times))
        starting_time = rospy.Time.now()
        _ = [b.remove() for b in bars]
        counts, bins, bars = ax_times.hist(times, bins=num_bins)

rospy.Subscriber('stateResult', Int32, times_cb)
fig_times.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Robot position and orientation

In [3]:
fig_pos = plt.figure(4)
ax_pos = fig_pos.add_subplot()

plot_odom, = ax_pos.plot([], [], 'ro')

ax_pos.set_title("Robot position")
ax_pos.set(xlabel="X position", ylabel="Y position")

x_data, y_data = [] , []
arrow_module = 1
yaw = 0

def odom_callback(msg):
    global x_data, y_data, yaw
    
    x_data.append(msg.pose.pose.position.x)
    y_data.append(msg.pose.pose.position.y)
    
    orientation_q = msg.pose.pose.orientation
    orientation_list = [orientation_q.x, orientation_q.y, orientation_q.z, orientation_q.w]
    
    (roll, pitch, yaw) = euler_from_quaternion(orientation_list)

def plot_init():
    
    ax_pos.set_xlim(-5, 5)
    ax_pos.set_ylim(-5, 5)
    plot_odom.set_data([],[])
    
    return plot_odom, arrow

def update_plot(frame):
    plot_odom.set_data(x_data, y_data)
    if len(x_data) > 0:
        x = x_data[len(x_data)-1]
        y = y_data[len(y_data)-1]

        arrow.set_position((x,y))
        arrow.xy = (x+arrow_module*math.cos(yaw), y+arrow_module*math.sin(yaw))

    return plot_odom, arrow

rospy.Subscriber('/odom', Odometry, odom_callback)

arrow = ax_pos.annotate('',
                xy=(arrow_module*math.cos(0), arrow_module*math.sin(0)),
                xytext=(0,0),
                arrowprops=dict(facecolor='red', shrink=0.05)
               )

ani = FuncAnimation(fig_pos, update_plot, init_func=plot_init)
fig_pos.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …